# Welcome

####This is a Colab notebook for photo translation using GAN
####It is created and trained by Shaoanlu
####We have managed it fit our need and demostration


## Clone git and download weight/model

In [1]:
!git clone --recursive https://github.com/shaoanlu/fewshot-face-translation-GAN.git

Cloning into 'fewshot-face-translation-GAN'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 971 (delta 33), reused 41 (delta 18), pack-reused 910
Receiving objects: 100% (971/971), 13.24 MiB | 52.36 MiB/s, done.
Resolving deltas: 100% (174/174), done.
Submodule 'face_toolbox_keras' (https://github.com/shaoanlu/face_toolbox_keras.git) registered for path 'face_toolbox_keras'
Cloning into '/content/fewshot-face-translation-GAN/face_toolbox_keras'...
remote: Enumerating objects: 43, done.        
remote: Counting objects: 100% (43/43), done.        
remote: Compressing objects: 100% (31/31), done.        
remote: Total 273 (delta 18), reused 29 (delta 11), pack-reused 230        
Receiving objects: 100% (273/273), 247.62 MiB | 38.49 MiB/s, done.
Resolving deltas: 100% (122/122), done.
Submodule path 'face_toolbox_keras': checked out 'fb472da7d6138845930bb86ca1993f5aaac6d78e'


In [2]:
# There are import errors under keras == 2.2.5
!pip install keras==2.2.4

     |████████████████████████████████| 317kB 2.9MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


In [3]:
%cd "fewshot-face-translation-GAN"

/content/fewshot-face-translation-GAN


In [4]:
# Download pre-trined weights
!gdown https://drive.google.com/uc?id=1DUMmZGTGKMyEYSKy-w34IDHawVF24rIs
!gdown https://drive.google.com/uc?id=1xl8cg7xaRnMsyiODcXguJ83d5hwodckB

Downloading...
From: https://drive.google.com/uc?id=1DUMmZGTGKMyEYSKy-w34IDHawVF24rIs
To: /content/fewshot-face-translation-GAN/encoder.h5
6.26MB [00:00, 54.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1xl8cg7xaRnMsyiODcXguJ83d5hwodckB
To: /content/fewshot-face-translation-GAN/decoder.h5
124MB [00:00, 172MB/s]


In [0]:
!mkdir weights
!mv decoder.h5 weights/decoder.h5
!mv encoder.h5 weights/encoder.h5

## Load GAN model

In [6]:
import warnings
warnings.filterwarnings("ignore")

from models import FaceTranslationGANInferenceModel

model = FaceTranslationGANInferenceModel()

Using TensorFlow backend.














Found checkpoints in weights folder. Built model with pre-trained weights.


## Load face-toolbox_keras

In [7]:
from face_toolbox_keras.models.verifier.face_verifier import FaceVerifier
fv = FaceVerifier(classes=512)

from face_toolbox_keras.models.parser import face_parser
fp = face_parser.FaceParser()

from face_toolbox_keras.models.detector import face_detector
fd = face_detector.FaceAlignmentDetector()

from face_toolbox_keras.models.detector.iris_detector import IrisDetector
idet = IrisDetector()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


## Translate faces and download generated image

In [0]:
import numpy as np
from utils import utils
from matplotlib import pyplot as plt
from google.colab import files

In [0]:
# upload images to colab '/content/fewshot-face-translation-GAN' manually, as colab file upload fails often
fn_src = "man1.jpg"
fns_tar = ["man2.jpg"]

In [0]:
# run inferece code and download output images to local device
for i in range(len(fns_tar)):
  src, mask, aligned_im, (x0, y0, x1, y1), landmarks = utils.get_src_inputs(fn_src, fd, fp, idet)
  tar, emb_tar = utils.get_tar_inputs(fns_tar[i], fd, fv)
  out = model.inference(src, mask, tar, emb_tar)
  # cropped image of face area only
  result_face = np.squeeze(((out[0] + 1) * 255 / 2).astype(np.uint8))
  # assembled to original image
  result_img = utils.post_process_result(fn_src, fd, result_face, aligned_im, src, x0, y0, x1, y1, landmarks)
  name = fns_tar[i] + "+ man1" + ".png"  # change "man1" to your src image name
  plt.imsave(name, result_img)
  files.download(name)


In [0]:
# show generated image

plt.imshow(result_img)

## Download video from drive


In [8]:
!gdown --id 1nDk821lAuxTM36P7ZB_BDGs1ql67Wzli

Downloading...
From: https://drive.google.com/uc?id=1nDk821lAuxTM36P7ZB_BDGs1ql67Wzli
To: /content/fewshot-face-translation-GAN/jim_long_30fps.mov
7.81MB [00:00, 68.4MB/s]


In [9]:
!gdown --id 1-2scAAC4n5VRHq9GJrrVgR_8Yno0m8XE

Downloading...
From: https://drive.google.com/uc?id=1-2scAAC4n5VRHq9GJrrVgR_8Yno0m8XE
To: /content/fewshot-face-translation-GAN/seth_long_30fps.mov
4.67MB [00:00, 129MB/s]


In [0]:
!mkdir pictures
!mkdir pictures/seth
!mkdir pictures/jim

In [11]:
# extract frame from [seth.mov]
# runs slowly, please wait if you are testing our code
!ffmpeg -i seth_long_30fps.mov -vf fps=30 pictures/seth/seth%06d.png

ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena

In [12]:
# extract frame from [jim.mov]
# runs slowly, please wait if you are testing our code
!ffmpeg -i jim_long_30fps.mov -vf fps=30 pictures/jim/jim%06d.png

ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena

In [0]:
#!pip install ffmpeg-python

## Translate faces for video

In [0]:
from utils import utils
import numpy as np
from matplotlib import pyplot as plt

In [14]:
# add extracted image to lists
from PIL import Image
import glob
image_list_A = []
image_list_B = []
for filename in glob.glob('/content/fewshot-face-translation-GAN/pictures/jim/*.png'): #assuming png
    #im=Image.open(filename)
    image_list_A.append(filename)

for filename in glob.glob('/content/fewshot-face-translation-GAN/pictures/seth/*.png'): #assuming png
    #im=Image.open(filename)
    image_list_B.append(filename)

image_list_A.sort()
image_list_B.sort()

print(len(image_list_A))
print(len(image_list_B))

1300
1319


In [15]:
!mkdir converted
%cd converted

/content/fewshot-face-translation-GAN/converted


In [16]:
# translating face frame by frame
# for a 30-fps video, you can see the result when having about 300 conveted video
# because of different number of frames of each video, loop might stop if listA size is smaller

for i in range(len(image_list_B)):
  fns_tar = []
  fn_src = image_list_B[i]
  fns_tar.append(image_list_A[i])
  src, mask, aligned_im, (x0, y0, x1, y1), landmarks = utils.get_src_inputs(fn_src, fd, fp, idet)
  tar, emb_tar = utils.get_tar_inputs(fns_tar[0], fd, fv)
  out = model.inference(src, mask, tar, emb_tar)
  result_face = np.squeeze(((out[0] + 1) * 255 / 2).astype(np.uint8))
  result_img = utils.post_process_result(fn_src, fd, result_face, aligned_im, src, x0, y0, x1, y1, landmarks)
  name = "%06d"%i + ".png"

  plt.imsave(name, result_img)



Building FAN for landmarks detection...
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Done.
Multiple faces detected. Only the most confident face will be processed.
Multiple faces detected. Only the most confident face will be processed.
Multiple faces detected. Only the most confident face will be processed.
Multiple faces detected. Only the most confident face will be processed.
Multiple faces detected. Only the most confident face will be processed.
Multiple faces detected. Only the most confident face will be processed.
Multiple faces detected. Only the most confident face will be processed.


IndexError: ignored

In [18]:
# generate result video, using converted photos

!ffmpeg -framerate 30 -i %06d.png -vcodec libx264 -s 640x480 -pix_fmt yuv420p movie.mp4

# you can download 'movie.mp4' once process finished
from google.colab import files
files.download('movie.mp4')

ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena

## Translate face and show generated results [original by author]

In [0]:
src, mask, aligned_im, (x0, y0, x1, y1), landmarks = utils.get_src_inputs(fn_src, fd, fp, idet)
tar, emb_tar = utils.get_tar_inputs(fns_tar, fd, fv)

TypeError: ignored

In [0]:
out = model.inference(src, mask, tar, emb_tar)

In [0]:
# show input image
plt.figure()
plt.subplot(1,2,1)
plt.imshow(src)
plt.subplot(1,2,2)
plt.imshow(tar)

In [0]:
# show results

result_face = np.squeeze(((out[0] + 1) * 255 / 2).astype(np.uint8))
plt.imshow(result_face)

result_img = utils.post_process_result(fn_src, fd, result_face, aligned_im, src, x0, y0, x1, y1, landmarks)
plt.imshow(result_img)